In [ ]:
# Download the files from lichess
import os

if not os.path.exists("lichess/lichess_db_puzzle.csv.zst"):
    !wget https://database.lichess.org/lichess_db_puzzle.csv.zst
if not os.path.exists("lichess/lichess_db_puzzle.csv"):
    !zstd -d lichess/lichess_db_puzzle.csv.zst

In [ ]:
import pandas as pd
import chess

# Read CSV and keep only important columns
df = pd.read_csv("lichess/lichess_db_puzzle.csv")
df = df[["PuzzleId", "FEN", "Moves", "Rating", "Themes"]]

In [ ]:
def fen_play_first_move_and_force_white(fen, moves):
    board = chess.Board(fen)
    first_move = moves.split()[0]
    # Play the first move of the puzzle, actually setting it up
    board.push_uci(first_move)
    
    # Flip the board if it is black to move
    if board.turn == chess.BLACK:
        board.apply_mirror()
    
    # Return just the FEN of the board
    return board.board_fen()

df["white_to_play_FEN"] = df.apply(lambda r: fen_play_first_move_and_force_white(r["FEN"], r["Moves"]), axis=1)

In [ ]:
# Create one hot encoding of the themes
one_hots = df["Themes"].str.get_dummies(sep=" ")
df = df.join(one_hots)

# Save also
df.to_csv("lichess/lichess_db_puzzle_processed.csv", index=False)

In [1]:
import pandas as pd
df = pd.read_csv("lichess/lichess_db_puzzle_processed.csv")

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8-deep")
DIR = "../report/project/img/"

# Histogram of ratings
plt.hist(df["Rating"], bins=[n*25 for n in range(140)])
plt.ylabel("Count")
plt.xlabel(f"Glicko2 puzzle rating (min={df['Rating'].min()}, max={df['Rating'].max()})")
plt.axvline(df["Rating"].mean(), c="r", ls="--", label=f"Average rating, $\\mu={df['Rating'].mean():.0f}$")
plt.title("Histogram of lichess.org puzzle ratings")
plt.legend()
plt.savefig(DIR + "puzzle_histogram.png", dpi=250, bbox_inches="tight")
plt.close()

# Theme counts
all_themes = [(t, df[t].sum()) for t in df.columns
    if t not in ["PuzzleId", "FEN", "Moves", "Rating", "Themes", "white_to_play_FEN"]]

all_themes = sorted(all_themes, key=lambda x: x[1])
plt.figure().set_figheight(4.8*3)
plt.barh(*zip(*all_themes))
plt.ylabel(f"Theme (total themes={len(all_themes)})")
plt.xlabel("Count")
plt.title("Counts of all themes in the lichess.org puzzle database")
plt.savefig(DIR + "puzzle_theme_counts.png", dpi=250, bbox_inches="tight")
plt.close()

# Stacked histogram of some themes
_themes = ["backRankMate", "defensiveMove", "trappedPiece"]
for t in _themes:
    plt.hist(df[df[t] == 1]["Rating"],
                bins=[n*50 for n in range(70)],
                label=t,
                alpha=0.7)

plt.ylabel("Count")
plt.xlabel(f"Glicko2 puzzle rating (min={df['Rating'].min()}, max={df['Rating'].max()})")
plt.title("Histogram of lichess.org puzzle ratings with specific themes")
plt.legend()
plt.savefig(DIR + "puzzle_theme_histograms.png", dpi=250, bbox_inches="tight")
plt.close()

In [ ]:
# Chunk df into splits for HTCondor
n = 100
for i, r in enumerate(range(0, df.shape[0], n)):
    df.loc[r:r+n, ["PuzzleId", "white_to_play_FEN"]].to_csv(f"condor/puzzles_{i}.csv")

In [2]:
# 60/20/20 split train/val/test
train = df.sample(frac=0.6, random_state=1337)
val = df.drop(train.index).sample(frac=0.5, random_state=1338)
test = df.drop(train.index).drop(val.index)

train.to_csv("lichess/lichess_db_puzzle_train.csv")
val.to_csv("lichess/lichess_db_puzzle_val.csv")
test.to_csv("lichess/lichess_db_puzzle_test.csv")